In [5]:
import os
import mysql.connector
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline

In [10]:
user_name = os.environ.get('DB_USER')
password = os.environ.get('DB_PASSWORD')

In [11]:
DB_NAME = 'BacterialGrowth'

## Database functions

In [12]:
def execute(phrase):
    """This function create a connection to the database and execute a command.
    :param phrase: str SQL sentence to execute
    :return: list of str received from the database after command execution.
    """
    try:
        cnx = mysql.connector.connect(user=user_name, password=password,host='localhost',database=DB_NAME)
        cnx.get_warnings = True
        cursor = cnx.cursor()
        cursor.execute(phrase)
        res = []
        for row in cursor:
            res.append(row)

        warnings = cursor.fetchwarnings()
        if warnings: 
            for i in range(len(warnings)):
                print("Warning - "+warnings[i][2])
        cursor.close()
        cnx.commit()
        cnx.close()
        return res
    except mysql.connector.Error as err:
        print("Something went wrong: {}".format(err))

In [29]:
def getWhereClause(args):
    clause = "WHERE ("
    for key, val in args.items():
        clause = clause + key + "= '" + str(val) + "' AND "
    clause = clause[:-5] + ')'
    return clause

In [38]:
def getAllRecords(table, **args):
    phrase = "SELECT * FROM " + table
    if args:
        where_clause = getWhereClause(args)
        phrase = phrase+" "+where_clause
    print(phrase)
    res = execute(phrase)
    return res[0]

In [40]:
studies = getAllRecords('Study')
print(studies)

experiments = getAllRecords('Experiment', studyId=1)
print(experiments)

SELECT * FROM Study
(1, 'MyFirstStudy', 'bla')
{'studyId': 1}
SELECT * FROM Experiment WHERE (studyId= '1')
(101, 1, 1, 1, 1, 1, 'A', 1, 0, 1000000, 10, 5.0, 20.0, 0, '0', 'Work?')


In [25]:
phrase = "INSERT IGNORE INTO Bacteria bacteriaGenus, bacteriaSpecies, bacteriaStrain) VALUES ('genero', 'especie', 'linea')"

In [26]:
execute(phrase)

Something went wrong: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'bacteriaGenus, bacteriaSpecies, bacteriaStrain) VALUES ('genero', 'especie', 'li' at line 1
This is to check if it continues


# INSERT IGNORE and get LAST ID

In [69]:
phrase = "INSERT IGNORE INTO Bacteria (bacteriaGenus, bacteriaSpecies, bacteriaStrain) VALUES ('genero', 'especie', 'linea')"
phrase2 = "SELECT bacteriaId FROM Bacteria WHERE bacteriaGenus = 'B' AND bacteriaSpecies = 'julia' AND bacteriaStrain ='ABC'"


In [70]:
insert = execute(phrase)
print('insert', insert)
select = execute(phrase2)
print('select', select)

insert (1, [])
select (1, [(43,)])


In [51]:
def getFieldsValues(args):
    fields = "("
    values = "("
    for key, val in args.items():
        fields = fields + key + ','
        values = values + "'" +str(val) + "',"
    fields = fields[:-1] + ')'
    values = values[:-1] + ')'
    return [fields, values]

In [48]:
bact = {
    'bacteriaGenus': 'genero',
    'bacteriaSpecies': 'especie',
    'bacteriaStrain': 'linea'
}

In [52]:
#fields = getFieldsValues(bact)[0]
#values = getFieldsValues(bact)[1]
fields, values = getFieldsValues(bact)
print(fields)
print(values)

(bacteriaGenus,bacteriaSpecies,bacteriaStrain)
('genero','especie','linea')


In [55]:
table = 'Bacteria'
phrase = "INSERT IGNORE INTO " + table + fields + " VALUES " + values
print(phrase)

INSERT IGNORE INTO Bacteria(bacteriaGenus,bacteriaSpecies,bacteriaStrain) VALUES ('genero','especie','linea')


In [57]:
len(bact)

3

In [67]:
def getWhereClause(args):
    clause = "WHERE ("
    for key, val in args.items():
        clause = clause + key + "= '" + val + "' AND "
        print(clause)
    clause = clause[:-5] + ')'
    print('\n', clause)

In [68]:
getWhereClause(bact)

WHERE (bacteriaGenus= 'genero' AND 
WHERE (bacteriaGenus= 'genero' AND bacteriaSpecies= 'especie' AND 
WHERE (bacteriaGenus= 'genero' AND bacteriaSpecies= 'especie' AND bacteriaStrain= 'linea' AND 

 WHERE (bacteriaGenus= 'genero' AND bacteriaSpecies= 'especie' AND bacteriaStrain= 'linea')


In [80]:
phrase = "SHOW KEYS FROM Experiment WHERE Key_name = 'PRIMARY'"
res = execute(phrase)
res[1][0][4]

'experimentId'

-------

In [97]:
def addExperiment(expDescription):
    phrase = "INSERT INTO Experiment (experimentDescription) VALUES ('" +expDescription+ "')"
    last_id = execute(phrase)[0]
    return last_id

In [120]:
def addCultivation(cultId, cultDescription, expId):
    phrase = "INSERT IGNORE INTO CultivationConditions (cultivationId, cultivationDescription, experimentId) VALUES ('"+cultId+"','"+cultDescription+"','"+expId+"')"
    execute(phrase)
    return False

In [121]:
def addReplicate(repId, repDescription, cultId, file):
    phrase = "INSERT IGNORE INTO TechnicalReplicates (replicateId, replicateDescription, cultivationId, bacterialAbundanceFile) "\
                "VALUES ('"+repId+"','"+repDescription+"','"+cultId+"','"+file+"')"
    last_id = execute(phrase)[0]
    return last_id
    

In [110]:
def countRecords(table, field, value):
    phrase = "SELECT COUNT(*) FROM "+table+" WHERE "+field+" = " + value
    res = execute(phrase)
    count = res[1][0][0]
    return count

## Code

In [11]:
EXP_DESCRIPTION = 'This experiment was done in Brussels.'
CULT_DESCRIPTION = 'In this perturbation, we kept pH at 5'
REP_DESCRIPTION = 'Replicate number #'

In [135]:
# These paths will come from the user!!

files = open("/Users/julia/bacterialGrowth_thesis/IntermediateFiles/listOfFiles.list", "r").readlines()
files = list(map(lambda s: s.strip(), files))

In [136]:
files

['/Users/julia/bacterialGrowth_thesis/Data/BT_batch_mucin1/1/BT_batch_mucin1_1.txt',
 '/Users/julia/bacterialGrowth_thesis/Data/BT_batch_mucin1/2/BT_batch_mucin1_2.txt',
 '/Users/julia/bacterialGrowth_thesis/Data/BT_batch_mucin1/3/BT_batch_mucin1_3.txt',
 '/Users/julia/bacterialGrowth_thesis/Data/BT_batch_mucin1/4/BT_batch_mucin1_4.txt',
 '/Users/julia/bacterialGrowth_thesis/Data/BT_batch_mucin1/5/BT_batch_mucin1_5.txt',
 '/Users/julia/bacterialGrowth_thesis/Data/BT_batch_mucin1/6/BT_batch_mucin1_6.txt',
 '/Users/julia/bacterialGrowth_thesis/Data/BT_batch_mucin1/7/BT_batch_mucin1_7.txt',
 '/Users/julia/bacterialGrowth_thesis/Data/BT_batch_mucin1/8/BT_batch_mucin1_8.txt',
 '/Users/julia/bacterialGrowth_thesis/Data/BT_batch_mucin1/9/BT_batch_mucin1_9.txt']

In [140]:
def addNewExperiment(files):
    expId = addExperiment(EXP_DESCRIPTION)

    number_cult = countRecords('CultivationConditions', 'experimentId', str(expId))
    
    cultId = expId*100 + number_cult + 1
    
    addCultivation(str(cultId), CULT_DESCRIPTION, str(expId))
    
    number_rep = countRecords('TechnicalReplicates', 'cultivationId', str(cultId))
    
    for i, file in enumerate(files):
        print(i, file)
        repId = str(cultId) + '_' + str(number_rep + i + 1)
        addReplicate(repId, REP_DESCRIPTION, str(cultId), file)

In [141]:
addNewExperiment(files)

0 /Users/julia/bacterialGrowth_thesis/Data/BT_batch_mucin1/1/BT_batch_mucin1_1.txt
1 /Users/julia/bacterialGrowth_thesis/Data/BT_batch_mucin1/2/BT_batch_mucin1_2.txt
2 /Users/julia/bacterialGrowth_thesis/Data/BT_batch_mucin1/3/BT_batch_mucin1_3.txt
3 /Users/julia/bacterialGrowth_thesis/Data/BT_batch_mucin1/4/BT_batch_mucin1_4.txt
4 /Users/julia/bacterialGrowth_thesis/Data/BT_batch_mucin1/5/BT_batch_mucin1_5.txt
5 /Users/julia/bacterialGrowth_thesis/Data/BT_batch_mucin1/6/BT_batch_mucin1_6.txt
6 /Users/julia/bacterialGrowth_thesis/Data/BT_batch_mucin1/7/BT_batch_mucin1_7.txt
7 /Users/julia/bacterialGrowth_thesis/Data/BT_batch_mucin1/8/BT_batch_mucin1_8.txt
8 /Users/julia/bacterialGrowth_thesis/Data/BT_batch_mucin1/9/BT_batch_mucin1_9.txt


In [150]:
PROJECT_DIRECTORY = '/Users/julia/bacterialGrowth_thesis/'
experimentName = open(PROJECT_DIRECTORY + 'IntermediateFiles/experiments_info.txt', "r").readlines()[1]
experimentName

'BT_batch_mucin1_1.txt BT_batch_mucin1 1\n'

In [151]:
[x.split(' ')[1] for x in open(PROJECT_DIRECTORY + 'IntermediateFiles/experiments_info.txt').readlines()][1]

'BT_batch_mucin1'

## Variable number of arguments

In [177]:
def addExperiment(**args):
    fields = "("
    values = "("
    for key, val in args.items():
        fields = fields + key + ','
        values = values + "'" +val + "',"
    fields = fields[:-1] + ')'
    values = values[:-1] + ')'
    phrase = "INSERT INTO Experiment " + fields + " VALUES " + values
        
    last_id = execute(phrase)[0]
    return last_id

In [178]:
ph = addExperiment(experimentName='Hola', experimentDescription='Esto es una prueba')
ph

4